In [17]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.emb_auto import AutoDataset
from Model.AE import Auto

import utils

import matplotlib.pyplot as plt

In [18]:
# 데이터 다운 & 전처리

df = pd.read_feather('../데이터/기본데이터.feather')

try: del df['Unnamed: 0']
except: pass

In [19]:
# df[0] = df['0'].apply(utils.to_list)
# df[1] = df['1'].apply(utils.to_list)

In [20]:
df[0] = df['0']
df[1] = df['1']

In [21]:
len_df = len(df)
train_len = int(len_df *0.6)
val_len = int(len_df * 0.3)

batch_size = 64

train_dataset = AutoDataset(df[:train_len])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = AutoDataset(df[train_len:train_len+val_len])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = AutoDataset(df[train_len+val_len:])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [22]:
for x,y in train_loader:
  print("X 크기 : {}".format(x.shape))
  print("Y 크기 : {}".format(y.shape))
  break

X 크기 : torch.Size([64, 7])
Y 크기 : torch.Size([64, 7])


In [23]:
input_size = 70
Model = Auto(input_size)

In [24]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

model = Model.to(device)

print('작동하는지 실험')
basic_data = torch.rand((1,input_size))
model(basic_data)


cpu is available
작동하는지 실험


(tensor([[0.0000, 0.0085, 0.2088, 0.2850, 0.1163, 0.3466, 0.3625, 0.4141, 0.0000,
          0.0942, 0.0000, 0.0000, 0.1039, 0.4107, 0.1641, 0.0000]],
        grad_fn=<ReluBackward0>),
 tensor([[0.4599, 0.5318, 0.4573, 0.4808, 0.4560, 0.5054, 0.5408, 0.4897, 0.4738,
          0.5221, 0.4785, 0.4511, 0.5495, 0.4484, 0.4832, 0.4989, 0.4765, 0.5200,
          0.4518, 0.5383, 0.4843, 0.5019, 0.4758, 0.5687, 0.5140, 0.5548, 0.5066,
          0.4893, 0.5687, 0.5216, 0.5127, 0.5209, 0.4748, 0.4602, 0.4871, 0.5186,
          0.5335, 0.4708, 0.4927, 0.4363, 0.5470, 0.5224, 0.4681, 0.4651, 0.4420,
          0.4838, 0.4942, 0.5526, 0.5087, 0.4531, 0.5172, 0.5297, 0.5034, 0.4746,
          0.5345, 0.4918, 0.5180, 0.4597, 0.4846, 0.4932, 0.4640, 0.5386, 0.5003,
          0.4705, 0.4681, 0.4985, 0.4778, 0.4400, 0.4925, 0.5111]],
        grad_fn=<SigmoidBackward0>))

In [25]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 1
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [16]:
## 학습하는 코드를 짜봐!!!

train_n = len(train_loader)
val_n = len(val_loader)

# print(train_loader)
best_val_loss = np.inf

for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for inputs, labels in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함
        inputs = inputs.to(device)
        labels = labels.to(device)
        vec, out = model(inputs)
        loss = criterion(out, labels)

        optimizer.zero_grad() #
        loss.backward() # loss가 최소가 되게하는
        optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
        running_loss += loss.item() # 한 배치의 loss 더해주고,


    print(running_loss/train_n)

    with torch.no_grad():
            print("Calculating validation results...")
            model.eval()
            val_loss = 0
            for inputs, labels in val_loader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    vec, outs = model(inputs)

                    loss= criterion(outs, labels)
                    val_loss += loss.item()

            avg_val_loss = val_loss/val_n
            if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    stop_counter = 0            
                    torch.save(model.state_dict(), 'Checkpoint/embedding_best_model.pth')


            # if val_acc > best_val_acc:
            #     print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
            #     # torch.save(model.module.state_dict(), f"{save_dir}/best.pth")
            #     best_val_acc = val_acc
            # torch.save(model.module.state_dict(), f"{save_dir}/last.pth")
            print("val")
            print(
                f"loss: {val_loss/val_n:4.8} || "
            )
    print('======================================================')



epoch : 0
train


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x7 and 70x32)

### 더미데이터 생성

In [11]:
data = []
model = model.requires_grad_(False)
for i, (x,_) in enumerate(test_dataset):
    data.append(str(model(x)[0].tolist()))
    if i==100:
        break

In [12]:
pd.DataFrame(data).to_excel('../데이터/인베딩더미.xlsx')